# Transfer Learning: Model Homes

## Background 

One service that some real estate technology (or "proptech") companies like [Redfin](https://www.redfin.com/what-is-my-home-worth) or [Zillow](https://www.zillow.com/sellerlanding/pricingtool/) offer are estimates of property value, for just about every property in the United States. 

But how do they know what any given property is worth? The answer is that they apply ready-made models, feeding the individual characteristics of any house into that model, with the resulting output being the model's prediction of what that home is currently worth.

Just like those firms, in this activity, your job is to load a ready-made model built from modeling many thousands of different home prices in Los Angeles County, California. You'll then apply this model to the smaller market for homes in San Diego County, California. Because these markets are similar, yet somewhat different, you'll also apply what you know about transfer learning to make adjustments to your pre-loaded model so that it can be better tailored to the specifics of homes in San Diego County.
    
## Instructions

1. Load the model (`los_angeles_model.json`) and its weights (`los_angeles_model.h5`) from the Resources folder.

2. Use the `layers` attribute or `summary` function to count how many layers there are.

3. Read in the San Diego County data (`san_diego.csv`, then `train_test_split` that data.
    >Note: the `y` variable should be `pricePerSquareFoot`, and the `X` data should include `livingArea`,`bathrooms`,`bedrooms`,and `garageSpaces`.

4. Freeze the existing layers of the loaded model. Verify all layers are frozen by printing the `summary` of the model's architecture.

5. Create a new network which is an exact copy of this loaded model, except that the top layer of the original model is removed.

6. Replace those removed layers with one or two new layers (including the final output layer). Ensure that these new trainable layers are added by using the `summary` function on this revised model. 

7. Finallly, `compile` and `fit` this newly revised model to the new data.

In [1]:
import pandas as pd
import numpy as np

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from sklearn.preprocessing import StandardScaler

In [2]:
#Imports
from tensorflow.keras.models import model_from_json

# Load the model (`los_angeles_model.json`) and its weights (`los_angeles_model.h5`) from the Resources folder.

# load json and create model
file_path = ("../Resources/los_angeles_model.json")
with open("../Resources/los_angeles_model.json", "r") as json_file:
    model_json = json_file.read()
loaded_model = model_from_json(model_json)

# load weights into new model
file_path = "../Resources/los_angeles_model.h5"
loaded_model.load_weights(file_path)

Metal device set to: Apple M1 Max


2022-03-15 12:57:07.115038: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-15 12:57:07.115150: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Use the `layers` attribute or `summary` function to count how many layers there are
loaded_model.layers

In [4]:
# Read in the San Diego County data
df = pd.read_csv('../Resources/san_diego.csv')

In [5]:
df.head(1)

,id,stateId,countyId,cityId,country,datePostedString,is_bankOwned,is_forAuction,event,time,...,parking,garageSpaces,hasGarage,levels,pool,spa,isNewConstruction,hasPetsAllowed,homeType,county
0,92037-16835229,9,1393,54296,USA,7/13/21,0,0,Listed for sale,1.626130e+12,...,1,2,1,Two,0,1,0,0,CONDO,San Diego County


In [6]:
#The `y` variable should be `pricePerSquareFoot` 
#The `X` data should include `livingArea`,`bathrooms`,`bedrooms`,and `garageSpaces`
y = df['pricePerSquareFoot']
X = df[['livingArea','bathrooms','bedrooms','garageSpaces']]

In [7]:
# Split into training and testing windows
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [8]:
# Freeze the existing layers of the loaded model
for layer in loaded_model.layers[0:-1]:
    layer.trainable = False

In [9]:
# Verify all layers are frozen by printing the `summary` of the
# model's architecture.
loaded_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 2)                 10        
                                                                 
 dense_7 (Dense)             (None, 6)                 18        
                                                                 
 dropout_1 (Dropout)         (None, 6)                 0         
                                                                 
 dense_8 (Dense)             (None, 3)                 21        
                                                                 
 dense_9 (Dense)             (None, 1)                 4         
                                                                 
Total params: 53
Trainable params: 4
Non-trainable params: 49
_________________________________________________________________


In [10]:
# Create a new network which is an exact copy of this loaded model,
# except that the top layer of the original model is removed.
transfer_model = Sequential()
for layer in loaded_model.layers[:-1]: 
    transfer_model.add(layer)

In [11]:
# Replace those removed layers with one or two new layers
# (including the final output layer). 

# Add an additional layer
transfer_model.add(Dense(10, activation="relu"))
# Add the final output layer
transfer_model.add(Dense(1))

In [12]:
# Ensure that these new trainable layers are added by using
# the `summary` function on this revised model.
transfer_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 2)                 10        
                                                                 
 dense_7 (Dense)             (None, 6)                 18        
                                                                 
 dropout_1 (Dropout)         (None, 6)                 0         
                                                                 
 dense_8 (Dense)             (None, 3)                 21        
                                                                 
 dense (Dense)               (None, 10)                40        
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 100
Trainable params: 51
Non-trainable par

In [13]:
# Compile the Sequential model
transfer_model.compile(loss="mean_absolute_error", optimizer="adam", metrics=["accuracy"])

In [14]:
# Fit the model
transfer_model.fit(X_train,y_train, 
                    epochs=20,
                    batch_size=100,
                    shuffle=True)

Epoch 1/20


2022-03-15 12:57:07.356703: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-15 12:57:07.509423: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


14/14 [==============================] - 0s 8ms/step - loss: 534.3995 - accuracy: 0.0872
Epoch 2/20
14/14 [==============================] - 0s 5ms/step - loss: 534.3602 - accuracy: 0.0864
Epoch 3/20
14/14 [==============================] - 0s 5ms/step - loss: 534.3181 - accuracy: 0.0849
Epoch 4/20
14/14 [==============================] - 0s 5ms/step - loss: 534.2745 - accuracy: 0.0857
Epoch 5/20
14/14 [==============================] - 0s 5ms/step - loss: 534.2277 - accuracy: 0.0849
Epoch 6/20
14/14 [==============================] - 0s 5ms/step - loss: 534.1814 - accuracy: 0.0849
Epoch 7/20
14/14 [==============================] - 0s 5ms/step - loss: 534.1309 - accuracy: 0.0667
Epoch 8/20
14/14 [==============================] - 0s 5ms/step - loss: 534.0781 - accuracy: 0.0144
Epoch 9/20
14/14 [==============================] - 0s 5ms/step - loss: 534.0249 - accuracy: 0.0061
Epoch 10/20
14/14 [==============================] - 0s 5ms/step - loss: 533.9661 - accuracy: 0.0015
Epoch 11/2